# Gerando Base de Dados dos Municípios de SP que elegeram um prefeito de alinhamento político diferente do segundo colocado

# Configurando o ambiente

In [35]:
import pandas as pd
import basedosdados as bd

## Importando a Base de Prefeitos Eleitos

In [36]:
query = """ SELECT * FROM `basedosdados.br_tse_eleicoes.resultados_candidato_municipio`
            WHERE ano = 2020
            AND sigla_uf = 'SP'
            AND cargo = 'prefeito' """

df_prefeitosSP = bd.read_sql(query, billing_project_id= "econometria-314719")

Downloading: 100%|█████████████████████████████████████████████████████████████| 2484/2484 [00:00<00:00, 2677.42rows/s]


# Criando Base para RDD

## Queries Prefeitos de SP eleitos no segundo turno de 2020






In [37]:
df_prefeitosSP.shape

(2484, 13)

In [38]:
df_prefeitosSP["id_municipio_tse"].unique().shape

(644,)

In [39]:
df_prefeitosSP.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,2020,1,eleicao ordinaria,SP,3500105,61018,prefeito,DEM,25,250001018914,1006655,eleito,11863
1,2020,1,eleicao ordinaria,SP,3500105,61018,prefeito,PSD,55,250001085198,1305529,nao eleito,3367
2,2020,1,eleicao ordinaria,SP,3500204,61034,prefeito,MDB,15,250001167312,418658,nao eleito,1217
3,2020,1,eleicao ordinaria,SP,3500204,61034,prefeito,PSDB,45,250001093392,927307,eleito,2087
4,2020,1,eleicao ordinaria,SP,3500303,61050,prefeito,PSDB,45,250000846054,1323085,eleito,9120


In [40]:
df_prefeitosSP.query("id_municipio_tse == '61387' | id_municipio_tse == '61298'")

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
49,2020,1,eleicao ordinaria,SP,3501509,61298,prefeito,PSDB,45,250000769354,810998,eleito,1447
665,2020,1,eleicao ordinaria,SP,3515129,61387,prefeito,PTB,14,250001182060,139853,eleito,1186


In [41]:
df_prefeitosSP[df_prefeitosSP["turno"] == 2].shape

(31, 13)

## Criando variável % de votos

In [43]:
total_votos2 = pd.DataFrame(df_prefeitosSP.groupby(["id_municipio_tse"]).votos.sum())

In [44]:
df_prefeitosSP = df_prefeitosSP.merge(total_votos2, how='inner', on='id_municipio_tse')

In [45]:
df_prefeitosSP.rename(columns = {"votos_x": "votos", "votos_y": "total_votos"}, inplace=True)

In [46]:
df_prefeitosSP["%votos"] = (df_prefeitosSP.votos / df_prefeitosSP.total_votos).round(5)

In [47]:
df_prefeitosSP.shape

(2484, 15)

In [48]:
df_prefeitosSP["id_municipio_tse"].unique().shape

(644,)

## Mantendo só o 1 e o 2 colocados para cada municipio (groupby) e depois criar uma coluna da diferença percentual entre eles

### Tentando usar nlargest()

In [49]:
grupo = pd.DataFrame(df_prefeitosSP.groupby(["id_municipio_tse"])["%votos"].nlargest(2))

In [50]:
grupo.head()

%votos
id_municipio_tse             
61000            685  0.72889
                 684  0.27111
61018            0    0.77892
                 1    0.22108
61026            337  0.70795

- Agora tenho que zerar o index do "Grupo"

In [51]:
grupo = grupo.reset_index()
grupo

,id_municipio_tse,level_1,%votos
0,61000,685,0.72889
1,61000,684,0.27111
2,61018,0,0.77892
3,61018,1,0.22108
4,61026,337,0.70795
...,...,...,...
1258,72435,2468,0.22833
1259,72451,2476,0.50875
1260,72451,2477,0.36096
1261,72737,2441,0.51726


In [52]:
grupo["id_municipio_tse"].unique().shape

(644,)

### Usando função .diff para subtrair uma linha pela outra 

In [53]:
dif_votos = grupo.groupby(["id_municipio_tse"]).diff(periods=-1)
dif_votos

,level_1,%votos
0,1.0,0.45778
1,NaN,NaN
2,-1.0,0.55784
3,NaN,NaN
4,-1.0,0.41590
...,...,...
1258,NaN,NaN
1259,-1.0,0.14779
1260,NaN,NaN
1261,-2.0,0.11481


### Juntando com a base df_prefeitosSP

In [54]:
dif_votos.drop(columns = "level_1", inplace = True)

In [55]:
dif_votos.rename(columns = {"%votos": "dif_votos"}, inplace = True)

In [56]:
grupo = grupo.join(dif_votos)

In [57]:
grupo = grupo.drop(columns = "level_1")

In [58]:
grupo.query("dif_votos <= 0.05")

,id_municipio_tse,%votos,dif_votos
8,61042,0.51833,0.04408
16,61085,0.38336,0.03343
22,61115,0.43730,0.02218
26,61131,0.36032,0.00292
29,61158,0.41087,0.04551
...,...,...,...
1203,71854,0.52346,0.04692
1208,71919,0.28880,0.02915
1233,72176,0.50183,0.00366
1235,72192,0.50588,0.01176


In [59]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(grupo[0:5])

  id_municipio_tse   %votos  dif_votos
0            61000  0.72889    0.45778
1            61000  0.27111        NaN
2            61018  0.77892    0.55784
3            61018  0.22108        NaN
4            61026  0.70795    0.41590


- Perceba que aqui o município 61387 só tem uma observação! O 61298 também. Ou seja, alguns municípios parecem que só tiveram 1 candidato! Tenho que verificar isso.
  - é isso mesmo! Vide notícia https://g1.globo.com/sp/sao-jose-do-rio-preto-aracatuba/eleicoes/2020/noticia/2020/09/30/quatro-cidades-do-noroeste-paulista-tem-apenas-um-candidato-a-prefeito.ghtml
- Se for isso, não tem problema eles serem apagados! Dado que o diferencial de votos não existiria, tornando-os inúteis para o RDD
- O problema é: Por que não estou vendo observações do 2 turno?

In [60]:
grupo["id_municipio_tse"].unique().shape

(644,)

### Juntando com base principal (df_prefeitosSP)

In [61]:
grupo.query('dif_votos.isnull()', engine='python')

,id_municipio_tse,%votos,dif_votos
1,61000,0.27111,NaN
3,61018,0.22108,NaN
5,61026,0.29205,NaN
7,61034,0.36834,NaN
9,61042,0.47425,NaN
...,...,...,...
1254,72397,0.29386,NaN
1256,72419,0.30382,NaN
1258,72435,0.22833,NaN
1260,72451,0.36096,NaN


In [62]:
grupo = grupo.dropna()
grupo.shape

(619, 3)

In [63]:
grupo = grupo.drop(columns="%votos")

In [64]:
df_prefeitosSP = df_prefeitosSP.merge(grupo, how = "inner", on = "id_municipio_tse")

In [65]:
df_prefeitosSP.head(5).T

,0,1,2,3,4
ano,2020,2020,2020,2020,2020
turno,1,1,1,1,1
tipo_eleicao,eleicao ordinaria,eleicao ordinaria,eleicao ordinaria,eleicao ordinaria,eleicao ordinaria
sigla_uf,SP,SP,SP,SP,SP
id_municipio,3500105,3500105,3500204,3500204,3500303
id_municipio_tse,61018,61018,61034,61034,61050
cargo,prefeito,prefeito,prefeito,prefeito,prefeito
sigla_partido,DEM,PSD,MDB,PSDB,PSDB
numero_candidato,25,55,15,45,45
sequencial_candidato,250001018914,250001085198,250001167312,250001093392,250000846054


In [66]:
df_prefeitosSP[df_prefeitosSP["turno"] == 2].shape

(31, 16)

In [67]:
df_prefeitosSP.to_csv("df_prefeitosSP.csv", sep = ";")

### Deixar só os 2 candidatos mais votados na base principal

In [68]:
df_prefeitosSP = df_prefeitosSP.sort_values(["%votos",'id_municipio_tse'],ascending=False).groupby('id_municipio_tse').head(2)

In [69]:
df_prefeitosSP.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,total_votos,%votos,dif_votos
985,2020,1,eleicao ordinaria,SP,3522505,65510,prefeito,PODE,19,250000635262,1704804,eleito,105494,106580,0.98981,0.97962
358,2020,1,eleicao ordinaria,SP,3508108,62634,prefeito,REPUBLICANOS,10,250000870308,1667613,eleito,6753,6922,0.97559,0.95118
1821,2020,1,eleicao ordinaria,SP,3543006,69612,prefeito,PP,11,250000893233,606804,eleito,5914,6123,0.96587,0.93174
1801,2020,1,eleicao ordinaria,SP,3542404,69493,prefeito,PSDB,45,250001140376,1486575,eleito,6120,6473,0.94547,0.89094
2388,2020,1,eleicao ordinaria,SP,3556008,72214,prefeito,DEM,25,250000986935,650432,eleito,4563,4898,0.93160,0.87688


## Criando Coluna "Governismo"

In [70]:
pd.Series(df_prefeitosSP["sigla_partido"].unique()).sort_values()

15           AVANTE
9         CIDADANIA
24               DC
4               DEM
7               MDB
27             NOVO
10         PATRIOTA
14          PC do B
13              PDT
8                PL
23              PMN
0              PODE
2                PP
21             PROS
22             PRTB
11              PSB
17              PSC
6               PSD
3              PSDB
12              PSL
18             PSOL
20               PT
5               PTB
25              PTC
16               PV
26             REDE
1      REPUBLICANOS
19    SOLIDARIEDADE
dtype: object

In [74]:
governismo = pd.read_csv("Aderencia_governista_2.csv", sep = ";")

In [75]:
governismo 

,Partido,Aderencia,Governismo
0,AVANTE,74,1
1,CIDADANIA,87,1
2,DEM,93,1
3,MDB,91,1
4,NOVO,92,1
5,PATRIOTA,94,1
6,PC do B,29,0
7,PDT,48,0
8,PL,90,1
9,PODE,77,1


In [76]:
governismo["Governismo"][15] = 0  # Colocando PSDB como Governismo = 0
governismo["Governismo"][15]

0

In [77]:
governismo.rename(columns = {"Partido": "sigla_partido"}, inplace = True)

In [78]:
df_prefeitosSP = df_prefeitosSP.merge(governismo, how = "inner", on = "sigla_partido")

In [79]:
df_prefeitosSP.query("id_municipio_tse == '62111'")

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,total_votos,%votos,dif_votos,Aderencia,Governismo
638,2020,1,eleicao ordinaria,SP,3505609,62111,prefeito,PTB,14,250000882397,1041847,nao eleito,6260,18184,0.34426,0.02865,90,1
967,2020,1,eleicao ordinaria,SP,3505609,62111,prefeito,PL,22,250000810374,917436,eleito,6781,18184,0.37291,0.02865,90,1


In [80]:
df_prefeitosSP.groupby(["id_municipio_tse"]).sum().query("Governismo == 1 & dif_votos <= 0.10").shape

(64, 8)

### Criar Coluna "RDD" para Primeiro Turno

In [81]:
teste = pd.DataFrame(df_prefeitosSP.groupby(["id_municipio_tse"]).Governismo.sum())

In [82]:
#teste["id_municipio_tse"] = teste.index

In [83]:
teste

,Governismo
id_municipio_tse,
61000,1
61018,2
61026,2
61034,1
61042,1
...,...
72397,2
72419,1
72435,1


In [84]:
teste.rename(columns = {"Governismo": "RDD"}, inplace=True)

In [85]:
df_prefeitosSP = df_prefeitosSP.merge(teste, how = "inner", on = "id_municipio_tse")

In [86]:
df_prefeitosSP.query("RDD == 1").shape

(601, 19)

In [87]:
df_RDD_1 = df_prefeitosSP.query("RDD == 1")

In [93]:
df_RDD_1.query("resultado == 'eleito' & dif_votos <= 0.05").shape

(64, 19)

# Salvando a base

In [89]:
df_RDD_1.to_csv("df_RDD.csv", sep = ";")

In [90]:
df_RDD_1.to_csv("df_RDD_2.csv", sep = ";", encoding= "UTF-8")

In [91]:
df_RDD_1.shape

(601, 19)